# Импорт

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import distance_metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

from joblib import dump


# Загружаем датасет и разбиваем её на выборку.

In [24]:
data = pd.read_csv('../../Data/DataLaba1/energy_task_moded.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('Appliances', axis=1), data['Appliances'], test_size=0.5, random_state=42)
X_train.shape, y_train.shape


((9867, 29), (9867,))

In [25]:
data

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,day,month,year,time
0,60.0,30.0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,6.60,733.5,92.000000,7.000000,63.000000,5.3,11.0,1.0,2016.0,1020.0
1,60.0,30.0,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,6.48,733.6,92.000000,6.666667,59.166667,5.2,11.0,1.0,2016.0,1030.0
2,50.0,30.0,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,6.37,733.7,92.000000,6.333333,55.333333,5.1,11.0,1.0,2016.0,1040.0
3,50.0,40.0,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,6.25,733.8,92.000000,6.000000,51.500000,5.0,11.0,1.0,2016.0,1050.0
4,60.0,40.0,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,6.13,733.9,92.000000,5.666667,47.666667,4.9,11.0,1.0,2016.0,1060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100.0,0.0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,22.70,755.2,55.666667,3.333333,23.666667,13.3,27.0,5.0,2016.0,1040.0
19731,90.0,0.0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,22.60,755.2,56.000000,3.500000,24.500000,13.3,27.0,5.0,2016.0,1050.0
19732,270.0,10.0,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,22.50,755.2,56.333333,3.666667,25.333333,13.3,27.0,5.0,2016.0,1060.0
19733,420.0,10.0,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,...,22.30,755.2,56.666667,3.833333,26.166667,13.2,27.0,5.0,2016.0,1070.0


# Подбор параметров в регрессоры.

In [3]:
ridge_optimal_for_line_regression = GridSearchCV(Ridge(), {'alpha': np.arange(0, 100, 1)}).fit(X_train, y_train)
ridge_optimal_for_line_regression.best_params_

{'alpha': 61}

In [4]:
lasso_optimal_for_line_regression = GridSearchCV(Lasso(), {'alpha': np.arange(0, 100, 1)}).fit(X_train, y_train)
lasso_optimal_for_line_regression.best_params_

c:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.357e+07, tolerance: 8.158e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one 

{'alpha': 0}

# Создание моделей

In [26]:
lr = LinearRegression()
lrl = Lasso(alpha=lasso_optimal_for_line_regression.best_params_['alpha'])
lrr = Ridge(alpha=ridge_optimal_for_line_regression.best_params_['alpha']).fit(X_train, y_train)

# Подбор гиппер-параметров для расфасовщика.

In [27]:
bagging_regressor_optimall_params = RandomizedSearchCV(BaggingRegressor(), 
                                                          {
                                                            'estimator': [lr, lrr, lrl],
                                                            'n_estimators' : np.arange(0, 20),
                                                            'bootstrap': [True, False]
                                                          },
                                                          n_iter=4,
                                                          n_jobs=4,
                                                          scoring='recall',
                                                          random_state=71
                                                          ).fit(X_train, y_train)
bagging_regressor_optimall_params.best_params_

c:\Users\Mike\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


{'n_estimators': 16, 'estimator': Ridge(alpha=61), 'bootstrap': True}

# Обучение

In [28]:
bagging_regressor = BaggingRegressor(
    estimator=bagging_regressor_optimall_params.best_params_['estimator'] if 'estimator' in bagging_regressor_optimall_params.best_params_ else None,
    n_estimators=bagging_regressor_optimall_params.best_params_['n_estimators'] if 'n_estimators' in bagging_regressor_optimall_params.best_params_ else 10,
    bootstrap=bagging_regressor_optimall_params.best_params_['bootstrap'] if 'bootstrap' in bagging_regressor_optimall_params.best_params_ else True
    )
bagging_regressor.fit(X_train, y_train)

BaggingRegressor(estimator=Ridge(alpha=61), n_estimators=16)

# Предсказание

In [29]:
predicted_bagging_regressor = bagging_regressor.predict(X_test)
predicted_bagging_regressor

array([ 37.59434415, 271.93323344,  44.05977154, ...,  62.59769678,
        42.84509298, 103.99296577])

# Оценка

In [30]:
errors = {'MAE': mean_absolute_error(predicted_bagging_regressor, y_test), 
          'RMSE': mean_squared_error(predicted_bagging_regressor, y_test),
          'MSE': mean_squared_error(predicted_bagging_regressor, y_test)**0.5,
          'MAPE': mean_absolute_percentage_error(predicted_bagging_regressor, y_test),
          'R^2': bagging_regressor.score(X_test, y_test)}
errors

{'MAE': 53.5534950889082,
 'RMSE': 9130.33586836791,
 'MSE': 95.55279100250243,
 'MAPE': 0.6668985811688058,
 'R^2': 0.151690625176913}